In [ ]:
# Subject:
# contains MIME-Version
# Begin Forwarded Message
# End Included Message
# Date (start)
# \tHome\t
# > on 
# References: <
# rosh.nair@avocadoit.com, (mail at start)
# Comments: To:
# reference <JSP-INTEREST
# List-Subscribe:
# List-Unsubscribe
# Return-Path:
# rikanth Raghavan <sraghavan@avocadoit.com>
# adrian.delagarza@pgcadvertising.com, patti.goodwyn@pgcadvertising.com,
# The time sheet called
# Received: by COFFEE
# n+ Summary
# To unsubscribe send a blank

# Synopsis:

In [1]:
from bertopic import BERTopic
from scipy.cluster import hierarchy as sch
import pandas as pd
import logging
from sklearn.metrics.pairwise import cosine_distances
from tqdm import tqdm
import numpy as np
from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
import ast
import re 
import sys
sys.path.append('../')
from utility.utility_functions import *

In [2]:
df_version_file = '../data/avocado_preprocessing_wip/avocado_parser_28_le_1000.parquet'

In [3]:
df = pd.read_parquet(df_version_file, engine='pyarrow')

## RegExs

In [ ]:
df = df[~df['text'].str.contains('Below is the result of your feedback form')]

In [ ]:
pt = r'^In-Reply-To:\s+<.*?>'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)

df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt= r'^X-Mailer: Internet Mail Service \([\d\.]{11}\)\nContent-Type: text\/plain(?:;\ncharset="iso-8859-1")?\n\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)

df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt= r'^Message-ID: <.*?@accenture\.com>'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)

df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'^Content-Type: text\/plain; .*?Message-ID: <.*?@.*?>\nX-OriginalArrivalTime: .*FILETIME=\[.*?:.*?\]'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)

df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'^Content-type:  text\/(?:html|plain)\\nContent-Description:.*?Content-Transfer-Encoding:  (?:quoted-printable\\nStatus:\s+(?:U|RO)\\nX-UIDL.*?|7bit)\n\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)

df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'^MIME-Version:.*?(?=\n\n)'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)

df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
df = df[~(df['text'].str.startswith('MIME-Version') & ~df['text'].str.contains('\n\n'))]
df = df[~(df['text'].str.startswith('MIME-version') & ~df['text'].str.contains('\n\n'))]

In [ ]:
pt = r'^Content-Type: multipart\/alternative.*?(?:Next)?(?:p|P)art.*?--\n\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)

df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'^X-Mailer.*?(?=--\n\n)'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)

df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'^X-Mailer.*?(?=--\n\n)'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)

df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'X-Mailer: ACCESS.*?(?=\n\n)'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)

df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))


In [ ]:
pt = r'^X-Mailer: AOL.*?--.*?"$'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)

df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'-+\s*Original Message\s*-+$'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)

df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'^Message-ID:.*?(?=\n\n)'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)

df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
df = df[~(df['text'].str.startswith('Message-ID') & ~df['text'].str.contains('\n\n'))]
df = df[~(df['text'].str.startswith('Message-id') & ~df['text'].str.contains('\n\n'))]
df = df[~(df['text'].str.startswith('Message-Id') & ~df['text'].str.contains('\n\n'))]

In [ ]:
pt = r'X-Mailer.*?(?=\n\n)'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)

df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
df = df[~(df['text'].str.startswith('X-Mailer') & ~df['text'].str.contains('\n\n'))]

In [ ]:
pt = r'^Content-Disposition.*?(?=\n\n)'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)

df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'^Content-Type:.*?(?=\n\n)'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)

df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))
df = df[~(df['text'].str.startswith('Content-Type:') & ~df['text'].str.contains('\n\n'))]

In [ ]:
pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)

df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'''^(?:")(?:\t|\').*<.*?>.*?(?=\n\n)'''


pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)

df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)

df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'^------_=_NextPart.*?(?=\n\n)'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)

df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))
pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)

df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'^This is a multi-part.*?\n\n(?:.*?NextPart.*?\n\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)

df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))
pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)

df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'^This is a multi-part.*?\n\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'^-+=_NextPart.*?(?=\n\n)'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)

df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt=r'^------=_NextPart_000_0002_01BDC145.51D0BB20 Content-Type: text/plain; charset="iso-8859-1" Content-Transfer-Encoding: 7bit'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)

df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
df = df[~df['text'].str.startswith('------=_NextPart')]
df = df[~df['text'].str.startswith('-----=_NEXT')]


In [ ]:
pt=r'^-----Original Task.*?(?=Requested by)'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))
pt=r'^Requested by.*?\n\n(?:\n------------\n)?'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))
pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)

df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
df = df[~df['text'].str.startswith('Requested by')]
df = df[~df['text'].str.startswith('------_=_')]

In [ ]:
df = df[~df['text'].str.contains('This is a multi-part message in MIME')]

In [ ]:
df = df[~(df['text'].str.contains('Error Message:') & df['text'].str.startswith('Script:'))]

In [ ]:
pt=r'^This message is in MIME format.*?\n\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
df = df[~( df['text'].str.startswith('------_') | df['text'].str.startswith('Message-') )]

In [ ]:
df = df[~( df['text'].str.startswith('Script: session') )]

In [ ]:
pt = r'^(?:--.*?\n\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
df = df[~( df['text'].str.startswith('--') & ~df['text'].str.contains('\n') & ~df['text'].str.contains('Airways') )]

In [ ]:
df = df[~ (df['text'].str.contains('The following addresses had permanent fatal errors') | df['text'].str.contains('Service unavailable') ) ]

In [ ]:
pt = r'^To:.*?(?=\n\n)' # Needs to be applied recursively
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
df = df[ ~( df['text'].str.startswith('To:') & ~ df['text'].str.contains('\n\n') ) ]

In [ ]:
df = df[~ (df['text'].str.contains(' newsletter') & df['text'].str.contains('unsubscribe') ) ]

In [ ]:
pt = r'^Content-Transfer-Encoding.*?(?=\n\n)'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'^Content-.*?(?=\n\n)'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
df = df[ ~(df['text'].str.startswith('Content-') & ~df['text'].str.contains('\n\n') ) ]

In [ ]:
pt = r'^(?:-+=+.*?\n\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
df = df[ ~(df['text'].str.startswith('--=') & ~df['text'].str.contains('\n\n')  ) ]

In [ ]:
pt = r'^X-.*?:.*?\n\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
df = df[ ~(df['text'].str.startswith('X-') & ~df['text'].str.contains('\n\n') ) ]

In [ ]:
pt = r'^(?:-+=+.*?\n\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'^(?:-+.*?Content.*?\n\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'^(?:-+[^\s]+)\n\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
df = df[~ (df['text'].str.startswith('--') & df['text'].str.contains('Content') & ~df['text'].str.startswith('--- Geeta Kottapalli') ) ]

In [ ]:
# Remove Massage Sign-Up Messages
df = df[~df['text'].str.startswith('Massage Sign-Up')]

In [ ]:
# Remove Error Messages
df = df[~df['text'].str.contains('memory_application crossed high limit')]
df = df[~df['text'].str.contains('Dear Amazon Customer')]
df = df[~df['text'].str.contains('E-card')]
df = df[~df['text'].str.contains('was delivered to the following recipient')]

In [ ]:
df = df[~df['text'].str.contains('Daily NewsBrief')]
df = df[~df['text'].str.contains('high-tech management news')]
df = df[~df['text'].str.contains('link2buy.com')]
df = df[~df['text'].str.contains('The following reply was made to PR')]
df = df[~df['text'].str.contains('ashampoo')]

In [ ]:
df = df[~df['text'].str.contains('Your fax which was sent to')]
df = df[~df['text'].str.contains('You have just received a new inbound')]
df = df[~df['text'].str.contains('If you would like to stop or modify')]
df = df[~df['text'].str.contains('If you wish to update your information or be removed from DCI')]

In [ ]:
df = df[~df['text'].str.contains('Following tests FAILED')]
df = df[~df['text'].str.contains('VentureWire')]
df = df[~df['text'].str.contains('Purchase Requisition\nForm')]
df = df[~df['text'].str.contains('Sender of the infected attachment')]

In [ ]:
df = df[~df['text'].str.contains('Norton AntiVirus found a virus')]
df = df[~df['text'].str.contains('McAfee.com eSecurity News')]
df = df[~df['text'].str.startswith('Thank you for registering')]
df = df[~(df['text'].str.startswith('Received:') & df['text'].str.endswith('>') )]

In [ ]:
df = df[~df['text'].str.contains('Connection refused by host')]
df = df[~df['text'].str.contains('Brainbench Test Alert')]
df = df[~df['text'].str.contains('No data received from host')]
df = df[~df['text'].str.contains('Please do not use the "Reply" button to respond to this email')]
df = df[~df['text'].str.contains('Synchronizing Mailbox')]
df = df[~df['text'].str.contains('Alerts on your MLX Profiles and Portfolios')]

In [ ]:
df = df[~df['text'].str.contains('THE INDUSTRY STANDARD')]
df = df[~df['text'].str.startswith('eFlash')]
df = df[~df['text'].str.contains('PostMasterDirect.com')]
df = df[~df['text'].str.startswith('NewsScan Daily')]
df = df[~df['text'].str.contains('The electronic newsletter')]
df = df[~df['text'].str.contains('PlanetFeedback.com offers')]
df = df[~df['text'].str.contains('WHAT\'S NEW NOW')]

In [ ]:
df = df[~df['text'].str.startswith('------------- Begin Forwarded Message')]
pt = r'(?:\n)*-+ Begin Forwarded Message -+$'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'(?:\n>)*-+ Begin Forwarded Message -+\n>$'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'(?:\n>)*-+ Begin Forwarded Message -+\n>$'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'^Subject:.*?\n\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
df = df[~(df['text'].str.startswith('WHAT\'S NEW NOW') & ~df['text'].str.contains('\n\n'))]

In [ ]:
pt = r'^MIME-Version.*?\n\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'^Return-Path:.*?\n\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
df =df[~(df['text'].str.startswith('''Return-Path:'''))]
df =df[~(df['text'].str.startswith('''From''') & df['text'].str.endswith('COFFEE>') )]

In [ ]:
pt = r'^Comments: To: .*?\n\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
df =df[~(df['text'].str.contains('''was not delivered to'''))]

In [ ]:
pt = r'^Mailing-List: (?:.*?@yahoogroups.com)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'^Date:.*?\n\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'^Date:.*?\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
df = df[~df['text'].str.startswith('Date:')]

In [ ]:
pt = r'^Priority:.*?\n\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
df = df[~df['text'].str.startswith('Priority:')]

In [ ]:
pt = r'^reference <JSP-INTEREST@www.javasoft.com>.*?\nTo: JSP-INTEREST@www.javasoft.com\n\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'^reference <JSP-INTEREST@www.javasoft.com>\nTo: JSP-INTEREST@www.javasoft.com\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'^In-Reply-To:.*?\n\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))


In [ ]:
pt = r'^In-Reply-To:.*?7bit\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
df =df[~(df['text'].str.startswith('In-Reply-To: Yolanda Vasquez'))]

In [ ]:
df =df[~(df['text'].str.startswith('Mime-Version'))]
df =df[~(df['text'].str.startswith('MIME-Version'))]
df =df[~(df['text'].str.startswith('Return-path:'))]
df =df[~(df['text'].str.startswith('Return Receipt\n'))]
df =df[~(df['text'].str.startswith('Return Receipt From'))]

In [ ]:
pt = r'Return-Receipt-To:.*?\n\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
df =df[~(df['text'].str.startswith('------=_'))]
df =df[~(df['text'].str.contains('You signed up to receive special offers'))] # newsletter
df =df[~(df['text'].str.startswith('Received:'))]

In [ ]:
pt = r'Mime-version:.*?\n\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))


In [ ]:
pt = r'Mime-version:.*?\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))


In [ ]:
df =df[~(df['text'].str.startswith('Mime'))]

In [ ]:
pt = r'Content.*?\n\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))


In [ ]:
df['text'] = df[~(df['text'].str.startswith('Content-') & ~df['text'].str.contains('7bit') )]

In [ ]:
pt = r'Return-.*?\n\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
df = df[~df['text'].str.startswith(r'Return-')]

In [ ]:
pt = r'^Date:.*?\n\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))


In [ ]:
pt = r'^Date:.*?\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))


In [ ]:
df = df[~df['text'].str.startswith('X-Mailer:')]
df = df[~df['text'].str.startswith('Content-Type:')]
df = df[~(df['text'].str.contains('Message-ID') & ~df['text'].str.contains('\n') )]

In [ ]:
pt = r'^Message-ID:.*?LocalDomain>\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'^Message-ID:.*?MCKINSEY.COM>\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
df = df[~(df['text'].str.startswith('Message-ID')  )]
df = df[~(df['text'].str.startswith('Message-Id')  )] 
df = df[~(df['text'].str.startswith('X-Sent-Mail')  )]

In [ ]:
pt = r'^> This message is in MIME format. Since your mail reader does not understand\nthis format, some or all of this message may not be legible.\n\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'^CC:.*?\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))


In [ ]:
df['text'] = df[~(df['text'].str.contains('??????????'  ,regex=False))]

In [ ]:
pt = r'^Message-.*?\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))


In [ ]:
df['text'] = df[~(df['text'].str.startswith('Message-'))]

In [ ]:
pt = r'^References:.*?\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
df['text'] = df[~(df['text'].str.startswith('References:'))]

In [ ]:
pt = r'^In-Reply-To:.*?\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'^From:.*?\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))
df['text'] = df[~(df['text'].str.startswith('From:'))]

In [ ]:
df = df[~(df['text'].str.contains('The Following Tests Failed!'))]

In [ ]:
pt = r'^Priority:.*?\n\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))


In [ ]:
df = df[~(df['text'].str.contains('ADVERTISEMENT='))]

In [ ]:
pt = r'^This is a MIME.*?\n\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
df = df[~(df['text'].str.startswith('Received:'))]
df = df[~(df['text'].str.startswith('The original message was received at'))]
df = df[~(df['text'].str.contains('subscribe for free'))]
df = df[~(df['text'].str.startswith('Return-Path:'))]

In [ ]:
pt = r'^boundary=.*?\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))


In [ ]:
pt = r'^List-Unsubscribe: <mailto:leave-stcetsig-l-34772P@stc.org>\nList-Subscribe: <mailto:subscribe-stcetsig-l@stc.org>\nList-Owner: <mailto:owner-stcetsig-l@stc.org>.*?\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))


In [ ]:
df = df[~(df['text'].str.startswith('List-Unsubscribe:'))] 

In [ ]:
pt = r'(?<=<<AvocadoIT_DoniSteward\.ppt>>).*'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'(?<=<<AvocadoIT uComm CHT 060601\.ppt>>).*'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'(?<=filename="GG-Wireless Application Magic Quadrant Market Note\.pdf).*'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'0M8R4KGxGuEAAAAAAAAAAAAAAAAAAAAAPgADAP7.*'   # removes powerpoints
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
df = df[~df['text'].str.contains('phonebooth.au')]

In [ ]:
df = df[~df['text'].str.contains('Componant /etc/default/login has been')]

In [ ]:
df = df[~df['text'].str.contains("Here's the list of files changed:")]

In [ ]:
df = df[~(df["text"].str.len()>20000)]

In [ ]:
def split_messages(text):
    pattern = r'At \d.*?wrote:'  # This matches: At <Date> <somebody> wrote:
    parts = re.split(pattern, text)
    return [part.strip() for part in parts if part.strip()]

# Create subset of DF that contain multi-messages, remove subset from the original DF
df_multi = df[(df['text'].str.contains('At ') & df['text'].str.contains('wrote:'))]
df = df[~(df['text'].str.contains('At ') & df['text'].str.contains('wrote:'))]

# Then collect all multi-messages in subset, explode the DF
df_multi['parts'] = df_multi['text'].apply(split_messages)
df_multi = df_multi.explode('parts')
df_multi = df_multi['parts'].to_frame()

#reassamble DFs
df_multi = df_multi.rename(columns={'parts': 'text'})
df = df['text'].to_frame()
df = pd.concat([df,df_multi])

In [ ]:
def remove_leading_gt(text):
    # Check if the text is a string (and not NaN or a float)
    if isinstance(text, str):
        # Split the text into lines
        lines = text.splitlines()
        
        # Check if every line starts with '>'
        if all(line.startswith('>') for line in lines):
            # Remove the '>' from the start of each line
            cleaned_lines = [line[1:].lstrip() for line in lines]
            # Join the lines back together into a single string
            return '\n'.join(cleaned_lines)
        else:
            # Return the text unchanged if not all lines start with '>'
            return text
    else:
        # If text is not a string, return it unchanged
        return text

In [ ]:
def split_and_remove_wrote(text):
    # Split the text into lines
    lines = text.splitlines()

    # Initialize a list to store the parts
    parts = []

    # Temporary list to accumulate lines for each part (before or after "wrote:")
    current_part = []

    for line in lines:
        if 'wrote:' in line:
            # If we encounter a "wrote:" line, push the current part to the list and reset it
            if current_part:
                # Join the accumulated lines with actual line breaks
                parts.append('\n'.join(current_part))  # Keep line breaks within the part
            current_part = []  # Reset for the next part
        else:
            # Accumulate the lines that are not "wrote:"
            current_part.append(line)
    
    # If there are any remaining lines in the current part, add them as the last part
    if current_part:
        parts.append('\n'.join(current_part))  # Keep line breaks in the final part

    return parts

df_multi = df[ (df['text'].str.contains('''wrote:''')) ]
df = df[ ~(df['text'].str.contains('''wrote:''')) ]

df_multi['parts'] = df_multi['text'].apply(split_and_remove_wrote)
df_multi = df_multi.explode('parts')
df_multi = df_multi['parts'].to_frame()

#reassamble DFs
df_multi = df_multi.rename(columns={'parts': 'text'})
df = df['text'].to_frame()
df = pd.concat([df,df_multi])

In [ ]:
df['text'] = df['text'].apply(remove_leading_gt)
df['text'] = df['text'].apply(remove_leading_gt)
df['text'] = df['text'].apply(remove_leading_gt)
df['text'] = df['text'].apply(remove_leading_gt)
df = df[df['text'] != '']

In [ ]:
df = df.dropna(subset=['text'])

In [ ]:
pt = r'^Mailing-List.*?Subject:.*?\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))
df = df[ ~(df['text'].str.startswith('''Mailing-List:''')) ]

In [ ]:
pt = r'^Content-Type.*?\n\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))
df = df[ ~(df['text'].str.startswith('''Content-Type''')) ]

In [ ]:
df = df[ ~(df['text'].str.startswith('''--''') & ~df['text'].str.contains('''\n\n''') ) ]
df = df[ ~(df['text'].str.startswith('''--''') & ~df['text'].str.contains('''had trouble''') ) ]

In [ ]:
pt = r'^--.*?\n\n'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'\n\n-+\s*Original\s*Appointment\s*-+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'\n\n-+\s*Original\s*Message-+\nFrom:.*'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'(?:\n>)+\s*-+Original\s*Message\s*-+(?:\n>)\s*From:.*'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'(?:\n)*-*Original\s*Message\s*Follows(?::)(?:\n)*-+$'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
def split_messages(text):
    pattern = r'Original Message Follows:'  # This matches: At <Date> <somebody> wrote:
    parts = re.split(pattern, text)
    return [part.strip() for part in parts if part.strip()]

In [ ]:
df_follows = df[ (  df['text'].str.contains('''Original Message Follows:''') ) ]
df =  df[ ~(  df['text'].str.contains('''Original Message Follows:''') ) ]

df_follows['parts'] = df_follows['text'].apply(split_messages)
df_follows = df_follows.explode('parts')
df_follows = df_follows['parts'].to_frame()

df_follows = df_follows.rename(columns={'parts': 'text'})
df = df['text'].to_frame()
df = pd.concat([df,df_multi])

In [ ]:
df['text'] = df['text'].apply(remove_leading_gt)
df['text'] = df['text'].apply(remove_leading_gt)
df['text'] = df['text'].apply(remove_leading_gt)
df['text'] = df['text'].apply(remove_leading_gt)
df = df[df['text'] != '']

In [ ]:
pt = r'To\s+Post\s+a\s+message,\s+send\s+it\s+to:\s+extremeprogramming@eGroups\.com(?:\n(?:>)?)*To\s+Unsubscribe,\s+send\s+a\s+blank\s+message\s+to:(?:\n(?:>)?)*\s*extremeprogramming-unsubscribe@eGroups\.com(?:\n(?:>)?)*Ad-free courtesy of objectmentor\.com(?:\n(?:>)?)*$'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'^Date: (?:\d|\w).*?(?=\n)'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'^Sender.*?(?=\n)'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'^(?:>|\n|\s)*Responding to.*?(?=\n)'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
df = df[~df['text'].str.startswith('Responding to')]

In [ ]:
pt = r'^\s\(added by.*?(?=\n)'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
df = df[~(df['text'].str.startswith(' (added by'))]
df = df[~(df['text'].str.startswith("\x1b"))]
df = df[~(df['text']=='')]

In [ ]:
pt = r'^Subject:.*?(?=\n\n)'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r'^Subject:.*?(?=\n)'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
df = df[~(df['text'].str.startswith("Subject:"))]
df = df[~(df['text'].str.startswith("Message-"))]
df = df[~(df['text'].str.startswith("--"))]

In [ ]:
# Removing Dates at the start of a string
pt = r'^\d{2}\/\d{2}\/\d{4}.*?(?=\n)'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))
df = df[~(df['text']=='')]

In [ ]:
# Removing Adressants at the start of a string

pt = r"^(?:(?:(?:'[\w\.]+@\w+\.\w{2,3}'|[\w\s\(\)\.\-\n]+);)(?:\s|\n)){2,100}.*?\n"
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))
df = df[~(df['text']=='')]

In [ ]:
# Removing Adressants at the start of a string

pt = r"^(?:(?:(?:'[\w\.]+@\w+\.\w{2,3}'|(?:')?[\w\s\(\)\.\-\\n\,]+(?:')?);)(?:\s|\n)){1,100}.*?\n"
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))
df = df[~(df['text']=='')]

In [ ]:
pt = r"^'[\w]+@[\w\.]+\.\w{2,3}'\n"
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))
df = df[~(df['text']=='')]

In [ ]:
pt = r"^[\w]+@[\w\.\-]+\.\w{2,3}\n"
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))
df = df[~(df['text']=='')]

In [ ]:
pt = r"^Sent:.*?To:.*?\n\n"
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))
df = df[~(df['text']=='')]
df = df[~(df['text'].str.startswith('Sent:'))]

In [ ]:
df = df[~(df['text'].str.startswith(r"Subject:") & ~df['text'].str.contains('>'))]
df = df[~(df['text'].str.startswith(r"amitabh.") )]

In [ ]:
pt = r"^From:.*?(?=\n)"
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
df = df[~(df['text'].str.contains(r"^From:",regex=True) )]
df = df[~(df['text'].str.contains(r"^Received:",regex=True) )]
df = df[~(df['text'].str.contains(r"^Sent:",regex=True) )]

In [ ]:
pt = r"^Reply-To:.*?(?=\n\n)"
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE|re.MULTILINE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
df = df[~(df['text'].str.contains(r"^Reply-To:",regex=True) )]
df = df[~(df['text'].str.contains(r"You have subscribed",regex=True) )]
df = df[~(df['text'].str.contains(r"^--",regex=True) )]

In [ ]:
df = df[~(df['text'].str.contains(r"^From -",regex=True,flags=re.IGNORECASE) )]
df = df[~(df['text'].str.contains(r"^return-path",regex=True,flags=re.IGNORECASE) )]

In [ ]:
df = df[~(df['text'].str.contains(r"^This multi-part",regex=True,flags=re.IGNORECASE) )]

In [ ]:
df = df[~(df['text'].str.contains(r"message-id",regex=True,flags=re.IGNORECASE) & df['text'].str.startswith('From ') )]

In [ ]:
pt = r"^Note: forwarded message attached."
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r"^Do you Yahoo.*?(?=\n\n)"
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
df = df[~(df['text'].str.contains(r"^Do you Yahoo",regex=True,flags=re.IGNORECASE) )]

In [ ]:
pt = r"^(?:(?:,|;)?(?:\s|\\n)?[\w\.]+@[\w\.\-]+)+.*?\n"     # Email Lists at the start of strings
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
df = df[~(df['text'].str.contains(r"^message-id",regex=True,flags=re.IGNORECASE) )]
df = df[~(df['text'].str.contains(r"^return-path",regex=True,flags=re.IGNORECASE) )]
df = df[~(df['text'].str.contains(r"^Reporting-MTA",regex=True,flags=re.IGNORECASE) )]
df = df[~(df['text'].str.contains(r"does not like recipient",regex=True,flags=re.IGNORECASE) )]
df = df[~(df['text'].str.contains(r"Hi. This is the qmail-send program",regex=True,flags=re.IGNORECASE) )]

In [ ]:
df = df[~(df['text'].str.contains(r"mailbox has exceeded",regex=True,flags=re.IGNORECASE) )]
df = df[~(df['text'].str.contains(r"This message could not be delivered",regex=True,flags=re.IGNORECASE) )]

In [ ]:
pt = r"^In a message dated.*?writes:\n"
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
df = df[~(df['text'].str.contains(r"^<\?xml version",regex=True,flags=re.IGNORECASE) )]

In [ ]:
pt = r"^<\?xml.*?>"
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r"^xml\-interest:.*?\n\n"
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r"^xml\-interest:.*?\n\n"
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
df = df[~(df['text'].str.contains(r"^xml-interest",regex=True,flags=re.IGNORECASE) )]
df = df[~(df['text'].str.contains(r"^--",regex=True,flags=re.IGNORECASE) )]

In [ ]:
pt = r"^Importance:.*?\n"
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
pt = r"^Importance :.*?\n"
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
df = df[~(df['text'].str.contains(r"^Importance:",regex=True,flags=re.IGNORECASE) )]
df = df[~(df['text'].str.contains(r"^(?:>)?received:",regex=True,flags=re.IGNORECASE) )]

In [ ]:
pt = r"^(?:>)*x-mailer:.*?\n"
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))
df = df[~(df['text'].str.contains(r"^(?:>)*x-mailer:",regex=True,flags=re.IGNORECASE) )]

In [ ]:
pt = r"^content-type:.*?\n"
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))
df = df[~(df['text'].str.contains(r"^content-type",regex=True,flags=re.IGNORECASE) )]
df = df[~(df['text'].str.contains(r"^boundary",regex=True,flags=re.IGNORECASE) )]

In [ ]:
pt = r"^charset=.*?\n"
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
df = df[~(df['text'].str.contains(r"^charset=",regex=True,flags=re.IGNORECASE) )]
df = df[~(df['text'].str.contains(r"^--",regex=True,flags=re.IGNORECASE) )]

In [ ]:
pt = r"^content-.*?\n"
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

In [ ]:
df = df[~(df['text'].str.contains(r"^content-",regex=True,flags=re.IGNORECASE) )]

In [ ]:
pt = r"This message is in MIME format. Since your mail reader does not understand\nthis format, some or all of this message may not be legible.*?\n"
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
df['text'] = df['text'].apply(lambda x: re.sub(pattern,"",x))

# WIP

In [ ]:
# Responding to
# Note from

In [ ]:
test_list = df['text']
test_list = test_list.tolist()
test_list.sort()

In [ ]:
test_list

In [12]:
#test = df[(df['text'].str.contains(r"^[\w]+@[\w\.\-]+\.\w{2,3}\n",regex=True))]['text']
test = df[(df['text'].str.contains(r"from:.*?>$",regex=True,flags=re.IGNORECASE|re.MULTILINE) )]['text']

In [ ]:
pt = r"This message is in MIME format. Since your mail reader does not understand\nthis format, some or all of this message may not be legible.*?\n"
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
test_res = test.apply(lambda x: re.sub(pattern,"",x))

pt = r'^(?:\n)+'
pattern = re.compile(pt,flags=re.DOTALL|re.IGNORECASE)
test_res = test_res.apply(lambda x: re.sub(pattern,"",x))

In [13]:
len(test)

761

In [14]:
for t in test:
    print(repr(t))

'Class newClass =\nClass.forName("oracle.jdbc.driver.OracleDriver").newInstance();\n\n\n\n\n\n>From: Ravi N <ravi.nallakukalla@WIPRO.COM>\n>Reply-To: A mailing list about Java Server Pages specification and\n>     reference <JSP-INTEREST@java.sun.com>'
'Hi Ramesh,\nI can suggest you some good policies.\n1. Use BufferedStreams to write to the file.\n2. You can use threads to allow the main servlet to do something while\nthings are being written.\n3. You can make the Query faster using PreparedStatements.\n\nHope this would be of some help.\n\nBye,\nJiger\n\n\n\n>From: "Ramesh, Kesav" <kesav.ramesh@EDS.COM>\n>Reply-To: "A mailing list for discussion about Sun Microsystem\'s Java\n>        Servlet API Technology." <SERVLET-INTEREST@java.sun.com>'
"which jdk is installed on the red hat box?  i think it usually defaults to\nthe kaffe jre, which means that when you invoke 'java' or 'javac' you are\nactually invoking 'kaffe'.  i dont think that is jdk1.2 compliant (i may be\nwrong).\n\ndownlo

In [ ]:
#Message-Id
#Subject
#received
# --
# content-type
# x-uidl
# resent
# x-
# reply-to

In [ ]:
for t in test_res:
    print(repr(t))

## Testing

In [ ]:
test = df['text']
test = test[ (test.str.contains('To Post a message, send it to:   extremeprogramming@eGroups.com\n\nTo Unsubscribe, send a blank message to: extremeprogramming-unsubscribe@eGroups.com\n\nAd-free courtesy of objectmentor.com') ) ]

In [ ]:
for t in test:
    print(repr(t))

In [ ]:
for t in test_res:
    print(repr(t))

### Save DF

In [ ]:
df = df[(df != "").all(axis=1)]

In [ ]:
df = pd.Series(list(set(df['text'].tolist())),name='text').to_frame()

In [ ]:
df = df.dropna(subset=['text'])

In [ ]:
df.to_parquet('../data/avocado_preprocessing_wip/avocado_parser_28_le_1000.parquet', engine='pyarrow')

In [ ]:
df.shape

In [ ]:
print(df.iloc[11]['text'])